# Подготовка датасета

## Импорт необходимых библиотек

Загрузим необходимые для работы библиотеки.

In [2]:
import pandas as pd
import os
import shutil
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import yaml

## Первичный анализ данных

In [3]:
# Зададим переменные с названиями изображений,входящих в train и test
train_imgs = pd.read_csv('/kaggle/input/waste-detection/train.csv')
test_imgs = pd.read_csv('/kaggle/input/waste-detection/test.csv')

# Проверим первые 5 строк
display(train_imgs.head())
display(test_imgs.head())

,file_name,bbox,category_id
0,000001.jpg,"[840.0, 0.0, 176.0, 124.0]",4.0
1,000001.jpg,"[612.0, 306.0, 383.0, 397.0]",2.0
2,000001.jpg,"[990.92, 551.0, 105.00000000000011, 186.0]",4.0
3,000002.jpg,"[1000.0, 614.0, 98.0, 178.0]",4.0
4,000002.jpg,"[605.0, 358.0, 402.0, 409.0]",2.0


,file_name
0,000003.jpg
1,000006.jpg
2,000013.jpg
3,000016.jpg
4,000019.jpg


Посмотрим основную информацию по датафреймам, которые содержались в csv файлах.

In [4]:
train_imgs.info()
test_imgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32265 entries, 0 to 32264
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    32265 non-null  object 
 1   bbox         32148 non-null  object 
 2   category_id  32148 non-null  float64
dtypes: float64(1), object(2)
memory usage: 756.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  2999 non-null   object
dtypes: object(1)
memory usage: 23.6+ KB


Мы можем видеть, что и в файле с изображениями для обучающей выборки, и в файле с изображениями для тестовой выборки имена файла имеют формат строки, что ожидаемо.  Наиболее странный формат у столбца `bbox`: он имеет формат строки. Превращение строки в список напишем внутри функции преобразования датасета в формат yolo.

Посмотрим, какие уникальные значения классов встречаются в тренировочных данных.

In [5]:
train_imgs['category_id'].sort_values().unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., nan])

Мы видим, что классы начинаются с 1. Формат датасета для Yolo предусматривает классы, начинающиеся с 0. Поправим это ниже.

In [6]:
train_imgs['category_id'] = train_imgs['category_id']-1
train_imgs['category_id'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., nan])

Для дальнейшей обработки значений ограничивающих рамок bbox удалим из строк, в которых они записаны квадратные скобки.

In [7]:
train_imgs['bbox'] = train_imgs['bbox'].str.strip('[]')
display(train_imgs.head())

,file_name,bbox,category_id
0,000001.jpg,"840.0, 0.0, 176.0, 124.0",3.0
1,000001.jpg,"612.0, 306.0, 383.0, 397.0",1.0
2,000001.jpg,"990.92, 551.0, 105.00000000000011, 186.0",3.0
3,000002.jpg,"1000.0, 614.0, 98.0, 178.0",3.0
4,000002.jpg,"605.0, 358.0, 402.0, 409.0",1.0


In [8]:
train_imgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32265 entries, 0 to 32264
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    32265 non-null  object 
 1   bbox         32148 non-null  object 
 2   category_id  32148 non-null  float64
dtypes: float64(1), object(2)
memory usage: 756.3+ KB


## Преобразование датасета в формат для модели YOLO

Зададим функцию для разделения датасета на тренировочную, валидационную и тестовую выборки для модели Yolo.

In [9]:
# Настройки
images_dir = '/kaggle/input/waste-detection/images'
output_dir = os.path.join('/kaggle/working/',"yolo_dataset")
os.makedirs(output_dir, exist_ok=True)

TRAIN_RATIO = 0.75
RANDOM_STATE = 42

In [10]:
# Зададим структуру папок для датасета
output_structure = {
    "train": {"images": os.path.join(output_dir, "images/train"), "labels": os.path.join(output_dir, "labels/train")},
    "val": {"images": os.path.join(output_dir, "images/val"), "labels": os.path.join(output_dir, "labels/val")},
    "test": {"images": os.path.join(output_dir, "images/test")},
}

# Создание папок
for split, paths in output_structure.items():
    for path in paths.values():
        os.makedirs(path, exist_ok=True)

In [11]:
# Функция преобразования bbox из формата CSV в YOLO
def convert_bbox_yolo(image_width, image_height, x_left, y_top, width, height):
    x_center = (x_left + width / 2) / image_width
    y_center = (y_top + height / 2) / image_height
    norm_width = width / image_width
    norm_height = height / image_height
    return x_center, y_center, norm_width, norm_height

In [12]:
# Создадим список названий файлов для обучения
train_files = train_imgs['file_name'].unique()
# Разделим файлы для обучения на тренировочную и валидационную выборку
train_files_train, train_files_val = train_test_split(train_files, train_size=TRAIN_RATIO, random_state=RANDOM_STATE)

In [13]:
# Функция для обработки данных
def process_split(files, split):
    for filename in tqdm(files, desc=f"Processing {split}"):
        # Копирование изображения
        src_image = os.path.join(images_dir, filename)
        dst_image = os.path.join(output_structure[split]["images"], filename)
        shutil.copy(src_image, dst_image)
        
        # Обработка аннотаций
        annotations = train_imgs[train_imgs['file_name'] == filename]
        if annotations.empty:
            continue  # Пропуск, если нет объектов
        
        # Открытие изображения с использованием Pillow
        with Image.open(src_image) as img:
            image_width, image_height = img.size
        
        label_file = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(output_structure[split]["labels"], label_file)
        with open(label_path, "w") as f:
            for _, row in annotations.iterrows():
                #bbox = row['bbox'].str.strip('[]') 
                #bbox = list(map(row['bbox'].split(',')))  # Преобразование bbox
                if pd.isna(row["bbox"]):
                    continue
                
                if isinstance(row["bbox"], str):  # Если bbox — строка
                    bbox = list(map(float, row["bbox"].split(",")))
                elif isinstance(row["bbox"], list):  # Если bbox уже список
                    bbox = row["bbox"]
                else:
                    raise ValueError(f"Unexpected bbox format: {row['bbox']}")
                    
                x_left, y_top, width, height = bbox
                class_id = row['category_id']
                x_center, y_center, norm_width, norm_height = convert_bbox_yolo(
                    image_width, image_height, x_left, y_top, width, height
                )
                f.write(f"{class_id} {x_center} {y_center} {norm_width} {norm_height}\n")

In [14]:
# Обработка тренировочной и валидационной выборок
process_split(train_files_train, "train")
process_split(train_files_val, "val")

# Обработка test.csv
for filename in tqdm(test_imgs['file_name'], desc="Processing test"):
    src_image = os.path.join(images_dir, filename)
    dst_image = os.path.join(output_structure["test"]["images"], filename)
    shutil.copy(src_image, dst_image)

print("Dataset preparation completed!")

Processing test: 100%|██████████| 2999/2999 [00:44<00:00, 67.39it/s]

Dataset preparation completed!


In [15]:
# Создадим файл конфигурации для модели в формате yaml
CLASSES = {
    'PET (transparent) (green)': 0,
    'PET (transparent) (brown)': 1,
    'PET (transparent) (blue)': 2,
    'PET (transparent)': 3,
    'PET (transparent) (dark blue)': 4,
    'PET (black)': 5,
    'PET (white)': 6,
    'PET (sticker)': 7,
    'PET (flacon)': 8,
    'PET (household chemicals)': 9,
    'PND (household chemicals)': 10,
    'PND packet': 11,
    'Other plastic': 12,
    'Other plastic (transparent)': 13,
    'Not plastic':14
}

idx_to_class = dict(zip(CLASSES.values(), CLASSES.keys()))
num_classes = 15

data_yaml = {
    'path': '/kaggle/input/garbage-detection-data/yolo_dataset/',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': num_classes,
    'names': idx_to_class
}

with open('data.yaml','w') as file:
    yaml.dump(data_yaml, file)